In [59]:
from pymongo import MongoClient
import requests
import json
import pandas as pd
import numpy as np
from getpass import getpass
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

### Conectar con la Collection en Mongo

In [60]:
def access_mdb_local_collection (database, collection):

    """ Function that returns a collection inside a DataBase 
    host in my local Server """

    # 1. Connect to my local Server
    client = MongoClient("localhost:27017")
    
    # 2. Access a DataBase:
    db = client[database]

    # 3. Acces a collection inside the given DataBase
    c = db.get_collection(collection)

    # 4. Return collection into a given DataBase host in a given Server
    return c

### Categories:

**10061**	Arts and Entertainment > Stadium > Basketball Stadium
    If nothign try: 10051	Arts and Entertainment > Stadium

**16033**	Landmarks and Outdoors > Park > Dog Park

**13377**	Dining and Drinking > Restaurant > Vegan 

**10032**	Arts and Entertainment > Night Club

**19031**	Travel and Transportation > Transport Hub > Airport
**19066**	Travel and Transportation > Train



**11030**	Business and Professional Services > Design Studio


In [61]:
markers = {
    10061: ["stadium" , "orange", "black", "baseball"],
    16033: ["dog_park", "green", "white", "dog"],
    13377: ["vegan_rest", "white", "green", "leaf"],
    10032: ["nigth_club", "black", "white", "champagne-glasses"],
    19066: ["transport", "blue", "white", "plane-departure"],
    19031: ["transport", "blue", "white", "plane-departure"],
    11030: ["dessing", "pink", "black", "pen-nib"]
    
}

# MAIN:

In [174]:
locations = { "Madrid" : "40.4193326,-3.6930825"}

In [175]:
c = access_mdb_local_collection ("ironhack", "madrid")

In [176]:
def extract_parameters(i):
            
            name = i["name"]
            
            lat = i["geocodes"]["main"]["latitude"]
            lon = i["geocodes"]["main"]["longitude"]
            
            #can have more than one:
            cat_list = [x["id"] for x in i["categories"]]
            
            try:
                city = i["location"]["locality"]
            except KeyError:
                city = np.nan
            
            params = {"city":city, "name": name, "cat_code":cat_list, "lat":lat, "lon":lon}
            
            return params

In [177]:
projection = { "_id":0,"categories":1, "geocodes":1,"name":1}
datos = list(c.find({},projection))

register = {"city": [], "place_name":[], "categories":[],"lat":[], "lon":[]}

for i in datos:
    
    info = extract_parameters(i)
    
    register["city"].append("madrid")
    register["place_name"].append(info["name"])
    register["categories"].append(info["cat_code"])
    register["lat"].append(info["lat"])
    register["lon"].append(info["lon"])

df = pd.DataFrame(register)

In [196]:
markers = {
    10051: ["stadium" , "orange", "black", "baseball"],
    16033: ["dog_park", "green", "white", "dog"],
    13377: ["vegan_rest", "white", "green", "leaf"],
    10032: ["nigth_club", "black", "white", "champagne-glasses"],
    19066: ["transport", "blue", "white", "plane-departure"],
    19031: ["transport", "blue", "white", "plane-departure"],
    11030: ["dessing", "pink", "black", "pen-nib"]
    
}

In [197]:
lista_conici = list(markers.keys())
lista_conici

[10051, 16033, 13377, 10032, 19066, 19031, 11030]

In [198]:
df.reset_index(inplace=True, drop=True)

In [205]:
def obtener_primera_coincidencia(categorias):
    for categoria in categorias:
        if categoria in lista_conici:
            return categoria
    return np.nan

# Aplicar la función a la columna 'Categorias' y crear una nueva columna 'Nueva_Columna'
df['match'] = df["categories"].apply(obtener_primera_coincidencia)

In [206]:
df

,city,place_name,categories,lat,lon,match
0,madrid,Barganzo,"[13286, 13377]",40.421086,-3.696823,13377.0
1,madrid,Gunilla Club,[10032],40.422582,-3.690987,10032.0
2,madrid,Giselle,[10032],40.422683,-3.691142,10032.0
3,madrid,Rubicon,[10032],40.422592,-3.689909,10032.0
4,madrid,Studio 42,"[10032, 13003]",40.422931,-3.690610,10032.0
...,...,...,...,...,...,...
254,madrid,Polideportivo Municipal de Orcasitas,[10051],40.375927,-3.712630,10051.0
255,madrid,Plaza de Toros de Vista Alegre,"[10051, 16026]",40.386375,-3.738919,10051.0
256,madrid,Iam Informatica del Ayuntamiento de Madrid,[10051],40.383820,-3.648450,10051.0
257,madrid,La Caja Mágica,[10051],40.368927,-3.684090,10051.0


In [207]:
def icon_set(category_code): # marker of categorye [10051]
    
    icon =  Icon(
            color = markers[category_code][1], #red
            icon_color = markers[category_code][2], #black
            icon = markers[category_code][3], #dog
            prefix="fa" # font-awesome website: fa
                )
            
    return icon

In [208]:
def city_markers(df, to_map):
    
        for index, row in df.iterrows():
            
            if df.iloc[index]["match"] == np.nan:
                continue
            else:
                category_code = df.iloc[index]["match"]
                #print(category_code)
                try:
                    #Set the icon for that kind of place:
                    icon_conf = icon_set(category_code)

                    #Extract position in witch locates:
                    lat = df.iloc[index]["lat"]
                    lon = df.iloc[index]["lon"]
                    coordenates = [lat, lon]

                    #Retrieve the category_code name (ej:"stadium")
                    # we know the position in the dictionary
                    lable = markers[category_code][0]

                    #Set the marker:
                    marker = Marker(coordenates, tooltip=lable ,  icon=icon_conf)

                    marker.add_to(to_map)
                
                
                except KeyError:
                    continue
        
        return

In [209]:
city_map = Map([40.4193326,-3.6930825], zoom_start=13,  tiles='CartoDB positron')

city_markers(df, city_map)

In [210]:
city_map

In [211]:
city_map.save("otros_madrid_tagsmap.html")

In [ ]:
punto:

40.4193367,-3.6956574